In [ ]:
# delete this block
import warnings
warnings.filterwarnings('ignore')
%autosave 0

# Introduction to spatial and spatiotemporal data in Python

This session will be used to go over the basics of accessing and manipulating spatial data with Python, using standard tools in the current Python geospatial ecosystem:
 - `rasterio` - for reading/writing raster data (among other things)
 - `numpy` - for manipulating N-dimensional arrays
 - `geopandas` - for reading/writing vector data and manipulating them in the form of a `pandas.DataFrame` with added functionality

Additionally, we will introduce some ease-of-use and performance oriented functionality implemented within `eumap`.

## Raster manipulation

First, we will open and inspect a raster with `rasterio`, a library that uses `GDAL` under the hood but provides an interface closer to idiomatic Python.

Let's gather some raster file paths.

In [ ]:

from pathlib import Path
from eumap.misc import find_files

data_home = Path.home()/'ODSE_workdir/data'
tile_id = 5606

tile_dir = data_home/f'tile_{tile_id}'

raster_paths = find_files(tile_dir, 'lcv/*.tif')

print('N files:', len(raster_paths))
print()
for rpath  in raster_paths[:10]:
    print(rpath)

We will open the second file on the list. This provides us with a `rasterio.DatasetReader` object.

In [ ]:

import rasterio as rio

raster = rio.open(raster_paths[1])
print(raster)

Aside from providing an interface for reading data, the `DatasetReader` contains useful metadata.

In [ ]:

print('driver:', raster.driver)
print('N bands:', raster.count)
print('shape:', raster.shape)
print('CRS:', raster.crs)
print('transform:', raster.transform)
print('nodata:', raster.nodata)

...all of which (and more) is available inside the `profile` property.

In [ ]:

print(raster.profile)

Calling the `read()` method with the band index as the argument (starting from 1) provides us with a `numpy` array containing the data.

In [ ]:

data = raster.read(1)
print(data, type(data))

As the data is read into a `numpy.ndarray`, we can perform the same operations as with any other `numpy` array, like computing statistics with the array's methods.

We can visualize the data with `eumap.plotter`, which provides a wrapper around `matplotlib` that ensures image aspect ratio is preserved and provides some added functionality, like transparency on `nodata`.

In [ ]:

from eumap import plotter
import numpy as np

# in case of plots not appearing properly in Jupyter
%matplotlib inline

data_min = data.min()
data_max = data.max()
data_median = np.median(data)

nodata = raster.nodata

plotter.plot_rasters(
    data,
    titles=f'min: {data_min}, max: {data_max}, median: {data_median}',
    figsize=5,
    nodata=nodata,
    vmin=int(data_min),
    vmax=int(data_max),
)

Aside from providing an easy way to deal with arrays and access to linear algebra, `numpy` supplies highly performant vectorized operations using BLAS libraries like MKL and OpenBLAS under the hood.

Let's find out where our data falls outside the interval from the 20th to the 80th percentile. Comparing an array with a number (or another array with compatible dimensions) yields an array of boolean elements. We can perform operations on these boolean arrays with Python's bitwise boolean operators.

In [ ]:

low, high = np.percentile(data, [20, 80])
print('P5:', low)
print('P95:', high)

hi_index = data > high
lo_index = data < low
nodata_index = hi_index | lo_index
print('index:', nodata_index)

pct_outside = 100 * nodata_index.sum() / nodata_index.size
print('% of data outside of bounds:', pct_outside.round(2))

Using the index we can now alter the out-of-bounds pixels to the nodata value.

In [ ]:

new_data = data.copy()
new_data[nodata_index] = nodata

plotter.plot_rasters(
    new_data,
    titles=f'nodata between 20th and 80th percentile',
    figsize=5,
    nodata=nodata,
    vmin=int(data_min),
    vmax=int(data_max),
)

...or clip the data to the interval.

In [ ]:

new_data[lo_index] = low
new_data[hi_index] = high

plotter.plot_rasters(
    new_data,
    titles=f'data clipped between 20th and 80th percentile',
    figsize=5,
    nodata=nodata,
    vmin=int(data_min),
    vmax=int(data_max),
)

Notice that the areas previously containing no data are now filled with valid values. That's because we didn't account for that nodata mask. We can produce an index by either comparing the data array to the nodata value, or better yet, using the `read_masks()` method of the `DatasetReader`. While nodata masks can sometimes be accounted for later on, it is beneficial to conserve resources by avoiding computation on nodata altogether.

In [ ]:

data_mask = raster.read_masks(1).astype(bool)
data_only = data[data_mask].copy()

hi_index = data_only > high
lo_index = data_only < low

data_only[lo_index] = low
data_only[hi_index] = high

new_data[:] = nodata
new_data[data_mask] = data_only

plotter.plot_rasters(
    new_data,
    titles=f'data clipped between 20th and 80th percentile',
    figsize=5,
    nodata=nodata,
    vmin=int(data_min),
    vmax=int(data_max),
)

Now let's open a folder and write our new data to a file. To open a file in write mode, we need to provide additional arguments, such as raster width and height in pixels, driver, etc. A transformation matrix and CRS are also required if we want our raster to be properly geospatially referenced. Luckily, all of this is contained in the `profile` `dict` of our `DatasetReader`. Since we have not changed any properties of the raster other than the data itself, we can pass the entire `profile` to the `DatasetWriter` as `**kwargs`.

Note that `rasterio` readers and writers can also be used as context managers (as one would use `open()` from the Python standard library).

In [ ]:
import os

out_dir = data_home/'session_2_outputs'
os.makedirs(out_dir, exist_ok=True)

out_path = out_dir/'raster.tif'

with rio.open(out_path, 'w', **raster.profile) as dst:
    print(dst)
    dst.write(new_data, 1)

`plotter.plot_rasters()` can also be called with dataset filepaths instead of data arrays. When used this way we do not have to provide the `nodata` argument, as it will be read from the file automatically.

In [ ]:

plotter.plot_rasters(
    out_path,
    figsize=5,
    vmin=int(data_min),
    vmax=int(data_max),
)

## Vector manipulation

For vector reading/writing and manipulation, we will use `geopandas`, a library that extends data frames with capabilities for processing geometries in a vectorized manner. `geopandas` internally uses `fiona` for I/O and `shapely` (and more recently `pygeos`) for handling geometry primitives.

Let's load the sample land cover points provided for the workshop and inspect the data.

In [ ]:

import geopandas as gp

points = gp.read_file(data_home/'land_cover_samples.gpkg')

print('CRS:', points.crs)
print('N points:', points.index.size)

points

We can compare the extent of the points with that of our sample raster. We can use `shapely` to construct polygons from the bounding box coordinates of both datasets. `shapely` interacts nicely with Jupyter and visualizes geometries on output.

In [ ]:

from shapely.geometry import box, MultiPolygon

raster_extent = box(*raster.bounds)
print('raster extent:', raster_extent)

points_bounds = points.cascaded_union.bounds
points_extent = box(*points_bounds)
print('points extent:', points_extent)

MultiPolygon([raster_extent, points_extent])

We can see the extent of the points is much larger than that of the raster, so we will utilize `geopandas` to perform a vectorized intersection check over the points. This produces a boolean array which we can use to index only the points inside the raster bounds.

In [ ]:

point_subset_index = points.intersects(raster_extent)

point_subset = points[point_subset_index]

print('N points:', point_subset.index.size)
point_subset.cascaded_union

We can now write our reduced point dataset to a file. Like `rasterio` (or `GDAL`), `geopandas` has support for various drivers. Here we will output the points to GeoJSON.

In [ ]:

point_subset.to_file(
    out_dir/'clipped_points.geojson',
    driver='GeoJSON'
)

## Computing a raster time series

Let's compute a spring NDVI timeseries for our tile from LANDSAT composites for the spring season for years 2000 to 2020. We can use `eumap.raster.read_rasters()` for a multithreaded read of multiple datasets into a single array. `read_rasters()` behaves in a time series friendly manner, stacking all layers into a multiband image. It also takes care of nodata masks, filling them with NaN.

In [ ]:

from eumap.raster import read_rasters

red_files = find_files(tile_dir, 'lcv/lcv_red_landsat.glad.ard_p50*03.21*.tif')
nir_files = find_files(tile_dir, 'lcv/lcv_nir_landsat.glad.ard_p50*03.21*.tif')

red, __ = read_rasters(raster_files=red_files)
nir, __ = read_rasters(raster_files=nir_files)

print('array shapes:', red.shape, nir.shape)

We can now use the stacked data to compute the entire NDVI series at once. We will then plot the series at a single pixel.

In [ ]:

import matplotlib.pyplot as plt
plt.style.use('seaborn')

ndvi = (nir - red) / (nir + red)

years = [*range(2000, 2021)]

def plot_series(data, index):
    xi, yi = index
    fig, ax = plt.subplots()
    ax.plot(years, data[yi,xi,:])
    ax.set_title(f'NDVI series at pixel {index}')
    ax.set_xticklabels(years, rotation=90)

plot_series(ndvi, (500, 500))

We can also use `plotter` to plot the series over the entire tile, but first we have to unstack the image into separate arrays for each year.

In [ ]:

ndvi = np.moveaxis(ndvi, -1, 0)
ndvi.shape

We will now plot the series over the last five years.

In [ ]:

plotter.plot_rasters(
    *ndvi.astype(np.float32)[-5:],
    figsize=10,
    titles=years[-5:],
    cmaps='Greens',
    vmin=-1,
    vmax=1,
)

Let's compute for each year the NDVI difference to the previous one and plot the results for the last five years.

In [ ]:

diff = ndvi[1:] - ndvi[:-1]

plotter.plot_rasters(
    *diff.astype(np.float32)[-5:],
    figsize=10,
    titles=years[-5:],
    cmaps='RdYlGn',
    vmin=-.5,
    vmax=.5,
)

We can use `save_rasters()` to batch write results for all the years in parallel, analogue to `read_rasters()`. `save_rasters()` takes series as multiband images, so we have to stack our `diff` array.

In [ ]:

from eumap.raster import save_rasters

out_files = [out_dir/f'ndvi_diff/ndvi_diff_{year}.tif' for year in years[1:]]

save_rasters(
    raster.name,
    out_files,
    np.stack(diff, -1).astype(np.float32),
    dtype='float32',
    nodata=-99.,
)

Let's overlay the results with our point subset. `eumap` provides a parallelized way of overlaying points with rasters.

In [ ]:

from eumap.mapper import SpaceOverlay
from datetime import datetime
overlay_points = point_subset

overlay = SpaceOverlay(
    point_subset[['geometry']],
    out_files,
).run()

Each point will now have attached the time series of NDVI differences.

In [ ]:

overlay

Let's plot the series at one of the points.

In [ ]:

def plot_series(point_id):
    diff_cols = sorted(overlay.columns[2:])
    fig, ax = plt.subplots()
    series = overlay.loc[point_id][diff_cols].values
    ax.plot(years[1:], series)
    ax.set_title(f'NDVI compared to previous year at point {point_id}')
    ax.set_xticklabels(years[1:], rotation=90)

plot_series(1990)

## Raster block processing

`rasterio` allows for rasters to be read from within a defined window.

In [ ]:

from rasterio.windows import Window

window = Window(
    col_off=0,
    row_off=0,
    width=5,
    height=5,
)

window_data = raster.read(1, window=window)

window_data

Cloud optimized GeoTIFFs are internally organized into blocks with local compression. Let's check the block number of our raster.

In [ ]:

block_windows = [*raster.block_windows()]

print('N blocks:', len(block_windows))

When reading a window from a raster, all blocks intersecting the window have to be read and decompressed. So if we read a single raster in parallel but only at windows corresponding to block boundaries, we minimize read time. `eumap.parallel.blocks` leverages this to enable efficient processing of large datasets, which allows for both faster processing on large hardware infrastructure and for long-running processing with limited resources.

We will compute NDVI for a single season within the boundary of our raster, but this time from pan-european LANDSAT mosaics hosted in S3 buckets.

To do this we first have to convert the geometry into the GeoJSON schema.

In [ ]:

from shapely.geometry import mapping

red_url = 'http://s3.eu-central-1.wasabisys.com/eumap/landsat/landsat_ard_20180625_20180912_red_p50.tif'
nir_url = 'http://s3.eu-central-1.wasabisys.com/eumap/landsat/landsat_ard_20180625_20180912_nir_p50.tif'

with rio.open(red_url) as src:
    print('raster size:', src.shape)

geometry = mapping(raster_extent)

print(geometry)

We will now initialize the reader and writer and define NDVI as a function.

In [ ]:

from eumap.parallel.blocks import RasterBlockReader, RasterBlockWriter

def calc_ndvi(red, nir):
    return (nir - red) / (nir + red)

reader = RasterBlockReader(reference_file=red_url)
writer = RasterBlockWriter(reader=reader)

We can now start the block-wise processing and write the result to a file.

In [ ]:

out_file = out_dir/'ndvi_blocks.tif'

writer.write(
    src_path=[red_url, nir_url],
    dst_path=out_file,
    geometry=geometry,
    block_func=calc_ndvi,
    nodata=-9999.,
    dtype='float32',
)

Let's check the results.

In [ ]:

plotter.plot_rasters(
    out_file,
    cmaps='Greens',
    figsize=5,
    vmin=-1,
    vmax=1,
)

## ODS data catalogue

`eumap.datasets.Catalogue` provides some abstraction over the ODS data catalogue. It provides a search utility to access dataset URLs.

In [ ]:

from eumap.datasets import Catalogue

cat = Catalogue(use_csw=False)
results = cat.search('ndvi')

results

We can ommit unwanted results with the `exclude` keyword argument

In [ ]:

results = cat.search('ndvi', exclude=['trend'])

results

...and also search by year.

In [ ]:

results = cat.search(
    'ndvi',
    exclude=['trend'],
    years=[2019]
)

results

Since the results behave more or less like regular Python strings, we can sort them into a time series and read with `read_rasters`. We will read the window corresponding to our test raster

In [ ]:
from rasterio.windows import from_bounds

results = sorted(results)

ref = rio.open(results[0])

window = from_bounds(
    *raster_extent.bounds,
    transform=ref.transform,
)

q_ndvi, __ = read_rasters(
    raster_files=[*map(str, results)],
    spatial_win=window,
    dtype=ref.profile['dtype'],
)

...and plot the results.

In [ ]:

plotter.plot_rasters(
    *np.moveaxis(q_ndvi, -1, 0),
    figsize=5,
    cmaps='Greens',
    nodata=ref.nodata,
    vmin=int(q_ndvi.min()),
    vmax=int(q_ndvi.max()),
)
